# Entrez-direct tool for searching the NCBI database

This link has instructions for installation and searching the databases. 
https://www.ncbi.nlm.nih.gov/books/NBK179288/ 


The first step is defining the search terms:

"Saccharum"[Organism] AND "biomol rna"[Properties] AND ("pathogen" OR "microbial" OR "microbiome" OR "fungus" OR "bacteria" OR "virus" OR "mycorrhizal" OR "endophyte" OR "rhizosphere") 

In [ ]:
# Encoded the query into URL terms:
# More information here https://www.ncbi.nlm.nih.gov/books/NBK25498/ 

import os

os.system('curl -X POST "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"  -d "db=sra&term=Saccharum%5BOrganism%5D%20AND%20%22biomol%20rna%22%5BProperties%5D%20AND%20(pathogen%20OR%20microbial%20OR%20microbiome%20OR%20fungus%20OR%20bacteria%20OR%20virus%20OR%20mycorrhiza%20OR%20endophyte%20OR%20rhizosphere)&retmax=400" > sugarcane_esearch_ids.txt')

sh: 1: curl: not found


32512

# Convert XML output into CSV - esummary

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from html import unescape

In [2]:
def parse_docsum(docsum):
    data = {}
    
    # Get the ID
    data['Id'] = docsum.findtext('Id')

    # Loop through all <Item> tags
    for item in docsum.findall('Item'):
        name = item.attrib.get('Name')
        value = item.text

        if name == 'ExpXml' and value:
            # Unescape and parse the embedded XML
            # exp_xml = ET.fromstring(unescape(value))
            exp_xml = ET.fromstring(f"<root>{unescape(value)}</root>")
            
            data['Title'] = exp_xml.findtext('.//Title')
            data['Instrument'] = exp_xml.find('.//Platform').attrib.get('instrument_model')
            stats = exp_xml.find('.//Statistics')
            data['Total Runs'] = stats.attrib.get('total_runs')
            data['Total Spots'] = stats.attrib.get('total_spots')
            data['Total Bases'] = stats.attrib.get('total_bases')
            data['Total Size'] = stats.attrib.get('total_size')

            submitter = exp_xml.find('.//Submitter')
            data['Submitter'] = submitter.attrib.get('acc')
            data['Center'] = submitter.attrib.get('center_name')
            data['Contact'] = submitter.attrib.get('contact_name')

            exp = exp_xml.find('.//Experiment')
            data['Experiment'] = exp.attrib.get('acc')

            study = exp_xml.find('.//Study')
            data['Study'] = study.attrib.get('acc')

            organism = exp_xml.find('.//Organism')
            data['Organism'] = organism.attrib.get('ScientificName')

            sample = exp_xml.find('.//Sample')
            data['Sample'] = sample.attrib.get('acc')

            lib = exp_xml.find('.//Library_descriptor')
            data['Library Strategy'] = lib.findtext('.//LIBRARY_STRATEGY')
            data['Library Source'] = lib.findtext('.//LIBRARY_SOURCE')
            data['Library Selection'] = lib.findtext('.//LIBRARY_SELECTION')
            data['Library Layout'] = 'SINGLE' if lib.find('.//SINGLE') is not None else 'PAIRED'

            data['Bioproject'] = exp_xml.findtext('.//Bioproject')
            data['Biosample'] = exp_xml.findtext('.//Biosample')

        elif name == 'Runs' and value:
            # run_xml = ET.fromstring(unescape(value))
            run_xml = ET.fromstring(f"<root>{unescape(value)}</root>")

            data['Run Accession'] = run_xml.attrib.get('acc')
            data['Run Spots'] = run_xml.attrib.get('total_spots')
            data['Run Bases'] = run_xml.attrib.get('total_bases')

        elif name == 'CreateDate':
            data['Create Date'] = value
        elif name == 'UpdateDate':
            data['Update Date'] = value

    return data

In [3]:
def xml_to_dataframe(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    records = [parse_docsum(docsum) for docsum in root.findall('DocSum')]
    df = pd.DataFrame(records)
    return df

In [7]:
df = xml_to_dataframe("sorghum_project_list_7November2025")
df

,Id,Title,Instrument,Total Runs,Total Spots,Total Bases,Total Size,Submitter,Center,Contact,...,Library Source,Library Selection,Library Layout,Bioproject,Biosample,Run Accession,Run Spots,Run Bases,Create Date,Update Date
0,35954038,Metabolomic and Transcriptomic Analysis of Dro...,Illumina NovaSeq 6000,1,20441684,6119255806,1882919550,SRA2001949,Xingjiang Academy of Agricultural Sciences,YUE Li,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415016,None,None,None,2024/11/05,2024/11/05
1,35954037,Metabolomic and Transcriptomic Analysis of Dro...,Illumina NovaSeq 6000,1,20358514,6095948666,1851542395,SRA2001949,Xingjiang Academy of Agricultural Sciences,YUE Li,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415015,None,None,None,2024/11/05,2024/11/05
2,35954036,Metabolomic and Transcriptomic Analysis of Dro...,Illumina NovaSeq 6000,1,20745508,6210300618,1893113189,SRA2001949,Xingjiang Academy of Agricultural Sciences,YUE Li,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415014,None,None,None,2024/11/05,2024/11/05
3,35954035,Metabolomic and Transcriptomic Analysis of Dro...,Illumina NovaSeq 6000,1,19444175,5820262192,1787584416,SRA2001949,Xingjiang Academy of Agricultural Sciences,YUE Li,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415013,None,None,None,2024/11/05,2024/11/05
4,35954034,Metabolomic and Transcriptomic Analysis of Dro...,Illumina NovaSeq 6000,1,21674601,6488969244,1982273902,SRA2001949,Xingjiang Academy of Agricultural Sciences,YUE Li,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415012,None,None,None,2024/11/05,2024/11/05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,734680,Drought-responsive small RNAs in Sorghum bicol...,Illumina Genome Analyzer IIx,1,19821595,693755825,374391067,ERA208090,"National Bureau of Plant Genetic Resources, Ne...",ArrayExpress,...,TRANSCRIPTOMIC,RANDOM,SINGLE,PRJEB1849,SAMEA2065629,None,None,None,2014/05/14,2014/08/12
457,734679,Drought-responsive small RNAs in Sorghum bicol...,Illumina Genome Analyzer IIx,1,22878203,800737105,449970914,ERA208090,"National Bureau of Plant Genetic Resources, Ne...",ArrayExpress,...,TRANSCRIPTOMIC,RANDOM,SINGLE,PRJEB1849,SAMEA2065628,None,None,None,2014/05/14,2014/08/12
458,734678,Drought-responsive small RNAs in Sorghum bicol...,Illumina Genome Analyzer IIx,1,21740738,760925830,415274994,ERA208090,"National Bureau of Plant Genetic Resources, Ne...",ArrayExpress,...,TRANSCRIPTOMIC,RANDOM,SINGLE,PRJEB1849,SAMEA1969582,None,None,None,2014/05/14,2014/08/12
459,734677,Drought-responsive small RNAs in Sorghum bicol...,Illumina Genome Analyzer IIx,1,18890943,661183005,373368895,ERA208090,"National Bureau of Plant Genetic Resources, Ne...",ArrayExpress,...,TRANSCRIPTOMIC,RANDOM,SINGLE,PRJEB1849,SAMEA2065467,None,None,None,2014/05/14,2014/08/12


In [8]:
df.to_csv("sorghum_project_list_7November2025.csv", index=False)

# Transform XML to df - efetch

it has the run accession

In [25]:
for pkg in root.findall('EXPERIMENT_PACKAGE'):
    print(pkg.find("SAMPLE").attrib.get("LIBRARY_LAYOUT"))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

# Load XML
tree = ET.parse('sorghum_runs.xml')
root = tree.getroot()

# List to hold records
data = []

# Loop through each EXPERIMENT_PACKAGE
for pkg in root.findall('EXPERIMENT_PACKAGE'):
    experiment = pkg.find('EXPERIMENT')
    run_set = pkg.find('RUN_SET')

    exp_id = experiment.attrib.get('accession')
    exp_alias = experiment.attrib.get('alias')
    title = experiment.findtext('TITLE')
    study_acc = experiment.find('STUDY_REF').attrib.get('accession')
    design_desc = experiment.findtext('DESIGN/DESIGN_DESCRIPTION')
    sample_acc = experiment.find('DESIGN/SAMPLE_DESCRIPTOR').attrib.get('accession')
    library_strategy = experiment.findtext('DESIGN/LIBRARY_DESCRIPTOR/LIBRARY_STRATEGY')
    instrument_model = experiment.findtext('PLATFORM/ILLUMINA/INSTRUMENT_MODEL')

    # Get library layout: SINGLE or PAIRED
    layout_elem = experiment.find('DESIGN/LIBRARY_DESCRIPTOR/LIBRARY_LAYOUT')
    if layout_elem is not None:
        if layout_elem.find('SINGLE') is not None:
            library_layout = 'SINGLE'
        elif layout_elem.find('PAIRED') is not None:
            library_layout = 'PAIRED'
        else:
            library_layout = 'UNKNOWN'
    else:
        library_layout = 'UNKNOWN'

    run = run_set.find('RUN') if run_set is not None else None
    run_acc = run.attrib.get('accession') if run is not None else None
    total_spots = run.attrib.get('total_spots') if run is not None else None
    total_bases = run.attrib.get('total_bases') if run is not None else None

    data.append({
        'experiment_accession': exp_id,
        'experiment_alias': exp_alias,
        'title': title,
        'study_accession': study_acc,
        'design_description': design_desc,
        'sample_accession': sample_acc,
        'library_strategy': library_strategy,
        'library_layout': library_layout,
        'instrument_model': instrument_model,
        'run_accession': run_acc,
        'total_spots': total_spots,
        'total_bases': total_bases
    })

# Create DataFrame
df = pd.DataFrame(data)
print(df.head())


  experiment_accession experiment_alias  \
0          SRX26611583           GL98D1   
1          SRX26611582           GL98C3   
2          SRX26611581           GL98C2   
3          SRX26611580           GL98C1   
4          SRX26611579          GL220B3   

                                               title study_accession  \
0  Metabolomic and Transcriptomic Analysis of Dro...       SRP543294   
1  Metabolomic and Transcriptomic Analysis of Dro...       SRP543294   
2  Metabolomic and Transcriptomic Analysis of Dro...       SRP543294   
3  Metabolomic and Transcriptomic Analysis of Dro...       SRP543294   
4  Metabolomic and Transcriptomic Analysis of Dro...       SRP543294   

  design_description sample_accession library_strategy library_layout  \
0      treated seeds      SRS23110265          RNA-Seq         PAIRED   
1      treated seeds      SRS23110264          RNA-Seq         PAIRED   
2      treated seeds      SRS23110263          RNA-Seq         PAIRED   
3      treated s

In [5]:
# Remove 51 miRNA-Seq projects
print(df["library_strategy"].value_counts())
df = df[df["library_strategy"] != 'OTHER']
df = df[df["library_strategy"] != 'ssRNA-seq']
df = df[df["library_strategy"] != 'miRNA-Seq']

library_strategy
RNA-Seq      430
OTHER         12
ssRNA-seq      9
ncRNA-Seq      6
miRNA-Seq      4
Name: count, dtype: int64


In [6]:
df.to_csv("sorghum_runs_cleaned.csv", index=False)

# Merge run and project information

In [1]:
import pandas as pd

df1 = pd.read_csv("sorghum_runs_cleaned.csv")
df2 = pd.read_csv("sorghum_project_list_7November2025.csv")

In [3]:
joined = df1.merge(df2, right_on='Experiment', left_on="experiment_accession", how='left')
joined = joined.drop(columns=['experiment_accession', 'experiment_alias', 'study_accession', 'instrument_model', 'total_spots', 'title', "Title", 'library_strategy', 'library_layout',
       'total_bases', 'Id', 'Instrument', 'Total Runs', 'Total Spots',
       'Total Bases'])
joined.to_csv("sorghum_runs_projects_merged_full.csv", index=False)

joined.columns

Index(['design_description', 'sample_accession', 'run_accession', 'Total Size',
       'Submitter', 'Center', 'Contact', 'Experiment', 'Study', 'Organism',
       'Sample', 'Library Strategy', 'Library Source', 'Library Selection',
       'Library Layout', 'Bioproject', 'Biosample', 'Run Accession',
       'Run Spots', 'Run Bases', 'Create Date', 'Update Date'],
      dtype='object')

In [5]:
joined

,design_description,sample_accession,run_accession,Total Size,Submitter,Center,Contact,Experiment,Study,Organism,...,Library Source,Library Selection,Library Layout,Bioproject,Biosample,Run Accession,Run Spots,Run Bases,Create Date,Update Date
0,treated seeds,SRS23110265,SRR31231318,1882919550,SRA2001949,Xingjiang Academy of Agricultural Sciences,YUE Li,SRX26611583,SRP543294,Sorghum bicolor,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415016,NaN,NaN,NaN,2024/11/05,2024/11/05
1,treated seeds,SRS23110264,SRR31231319,1851542395,SRA2001949,Xingjiang Academy of Agricultural Sciences,YUE Li,SRX26611582,SRP543294,Sorghum bicolor,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415015,NaN,NaN,NaN,2024/11/05,2024/11/05
2,treated seeds,SRS23110263,SRR31231320,1893113189,SRA2001949,Xingjiang Academy of Agricultural Sciences,YUE Li,SRX26611581,SRP543294,Sorghum bicolor,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415014,NaN,NaN,NaN,2024/11/05,2024/11/05
3,treated seeds,SRS23110262,SRR31231321,1787584416,SRA2001949,Xingjiang Academy of Agricultural Sciences,YUE Li,SRX26611580,SRP543294,Sorghum bicolor,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415013,NaN,NaN,NaN,2024/11/05,2024/11/05
4,treated seeds,SRS23110261,SRR31231322,1982273902,SRA2001949,Xingjiang Academy of Agricultural Sciences,YUE Li,SRX26611579,SRP543294,Sorghum bicolor,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415012,NaN,NaN,NaN,2024/11/05,2024/11/05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,Drought-responsive small RNAs in Sorghum bicol...,ERS239305,ERR266128,374391067,ERA208090,"National Bureau of Plant Genetic Resources, Ne...",ArrayExpress,ERX240648,ERP002522,Sorghum bicolor,...,TRANSCRIPTOMIC,RANDOM,SINGLE,PRJEB1849,SAMEA2065629,NaN,NaN,NaN,2014/05/14,2014/08/12
432,Drought-responsive small RNAs in Sorghum bicol...,ERS239304,ERR266129,449970914,ERA208090,"National Bureau of Plant Genetic Resources, Ne...",ArrayExpress,ERX240647,ERP002522,Sorghum bicolor,...,TRANSCRIPTOMIC,RANDOM,SINGLE,PRJEB1849,SAMEA2065628,NaN,NaN,NaN,2014/05/14,2014/08/12
433,Drought-responsive small RNAs in Sorghum bicol...,ERS239303,ERR266127,415274994,ERA208090,"National Bureau of Plant Genetic Resources, Ne...",ArrayExpress,ERX240646,ERP002522,Sorghum bicolor,...,TRANSCRIPTOMIC,RANDOM,SINGLE,PRJEB1849,SAMEA1969582,NaN,NaN,NaN,2014/05/14,2014/08/12
434,Drought-responsive small RNAs in Sorghum bicol...,ERS239302,ERR266130,373368895,ERA208090,"National Bureau of Plant Genetic Resources, Ne...",ArrayExpress,ERX240645,ERP002522,Sorghum bicolor,...,TRANSCRIPTOMIC,RANDOM,SINGLE,PRJEB1849,SAMEA2065467,NaN,NaN,NaN,2014/05/14,2014/08/12


In [4]:
joined.loc[:,['run_accession', 'Bioproject', "Library Strategy", "Library Layout"]].to_csv("sorghum_runs_projects_merged_filtered.csv", index=False)


,experiment_accession,experiment_alias,title,study_accession,design_description,sample_accession,library_strategy,library_layout,instrument_model,run_accession,...,Library Source,Library Selection,Library Layout,Bioproject,Biosample,Run Accession,Run Spots,Run Bases,Create Date,Update Date
0,SRX26611583,GL98D1,Metabolomic and Transcriptomic Analysis of Dro...,SRP543294,treated seeds,SRS23110265,RNA-Seq,PAIRED,Illumina NovaSeq 6000,SRR31231318,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415016,NaN,NaN,NaN,2024/11/05,2024/11/05
1,SRX26611582,GL98C3,Metabolomic and Transcriptomic Analysis of Dro...,SRP543294,treated seeds,SRS23110264,RNA-Seq,PAIRED,Illumina NovaSeq 6000,SRR31231319,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415015,NaN,NaN,NaN,2024/11/05,2024/11/05
2,SRX26611581,GL98C2,Metabolomic and Transcriptomic Analysis of Dro...,SRP543294,treated seeds,SRS23110263,RNA-Seq,PAIRED,Illumina NovaSeq 6000,SRR31231320,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415014,NaN,NaN,NaN,2024/11/05,2024/11/05
3,SRX26611580,GL98C1,Metabolomic and Transcriptomic Analysis of Dro...,SRP543294,treated seeds,SRS23110262,RNA-Seq,PAIRED,Illumina NovaSeq 6000,SRR31231321,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415013,NaN,NaN,NaN,2024/11/05,2024/11/05
4,SRX26611579,GL220B3,Metabolomic and Transcriptomic Analysis of Dro...,SRP543294,treated seeds,SRS23110261,RNA-Seq,PAIRED,Illumina NovaSeq 6000,SRR31231322,...,TRANSCRIPTOMIC,RANDOM,PAIRED,PRJNA1180449,SAMN44415012,NaN,NaN,NaN,2024/11/05,2024/11/05
